<a href="https://colab.research.google.com/github/VadimGrishin/CFR/blob/master/check_sites.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import csv
import requests
from lxml import html
import re

In [0]:
def csv_writer(data, path):
    """
    Write data to a CSV file path
    """

    with open(path, "w", newline='') as csv_file:
        writer = csv.writer(csv_file, delimiter=';')
        for line in data:
            writer.writerow(line)

In [0]:
def repl_utilwords(phrase):
    utlist = [
        'ОБЛ.', 'ПР-КТ', ' КРАЙ', 'УЛИЦА', 'ОБЛАСТЬ', 'Р-Н', 'ОКРУГ'
             ]
    utlist2 = [
        ' Г,', ' УЛ,', ' НАБ,', ' ШОССЕ,', ' ОБЛ,', ' ГОРОД,', ' РЕСП,', 
        ' СТ-ЦА,', ' ПГТ,', ' ПЕР,', 'ПЛ-КА', ' РЕСПУБЛИКА,'
             ]
    for u in utlist:
        phrase = phrase.replace(u,'').replace(' ,', ',').strip()
        
    for u in utlist2:
        phrase = phrase.replace(u,',').strip()
        
    return phrase

In [0]:
def parse_addr(addr):

    addr_keywords = []
    if addr:
        addr = repl_utilwords(addr[0]).strip() # .replace('Г ', '').replace('УЛ ', '')

        addr = re.sub(r'([\s,]+Д\s)', '#', addr)
        addr = re.sub(r'(ДОМ\s)', '#', addr)
        addr = re.sub(r'#(.*)', '', addr).strip() + ','

        # print(1, addr)

        addr = re.sub(r'([\s,]+\d+\s?\w?,)', ',', ' ' + addr)

        # print(2, addr)


        addr_keywords = re.findall(r'([\w/-]+),', addr)
        
        addr_keywords = [it for it in addr_keywords if len(it) > 2]
        
    return addr_keywords

In [0]:
def parse_orgname(orgname):
      orgname_keywords = re.findall(r'"(.*)"', orgname)
      
      x = []
      if orgname_keywords:

          x = orgname_keywords[0].replace(' -','-').replace('- ','-').replace('"', '').upper()

          x = x.split()
          small = ['ЭК', 'и', 'НП', '2', 'НН', 'СО', 'г.', 'ВО', 'ГТ', 'СК', '+', 
           'И', 'Эл', 'Т', '2', '№2', 'НА', '№1', 'ПО', 'ЭЛ', 'ГТ', 'И', 
           '14', 'В', 'Г.', 'Т', '№', 'КОМПАНИЯ', 'ОРГАНИЗАЦИЯ', 'ЗАВОД', 'КОМБИНАТ']
          
          for sm in small:
            try:
              x.remove(sm)
              x.remove(sm)
            except ValueError:
              pass
            
      return x

In [0]:
def filter_keywords2(o_kw2, o_kw):
    for kw in o_kw:
      try:
        o_kw2.remove(kw)
      except ValueError:
        pass
    
    return o_kw2
  

In [0]:
def csv_dict_reader(file_obj):
    """
    Read a CSV file using csv.DictReader
    """
    global data
    
    reader = csv.DictReader(file_obj, delimiter=';')
    for line in reader:
        
        inn = line['INN']
        name = line['Name']
        site1 = line['site1']
        site2 = line['site2']
        site_list_org = line['site_list_org']
        
        x = line['descr']
        
        orgname_keywords = parse_orgname(name)
        orgname_keywords2 = filter_keywords2(parse_orgname(x), orgname_keywords)
                
        y = re.findall(r'АДРЕС:(.*)', x)
        
        addr_keywords = parse_addr(y)
        
        
        data.append([inn, name, site1, site2, site_list_org, orgname_keywords, 
                     orgname_keywords2, addr_keywords])
        


        

In [0]:
data = [['INN', 'Name', 'site1', 'site2', 'site_list_org', 'orgname_keywords', 
         'orgname_keywords2', 'addr_keywords', 
         'site1_rates', 'site2_rates', 'site3_rates', 
         'site1_tot', 'site2_tot', 'site3_tot', 
         'site1_adr', 'site2_adr', 'site3_adr',
         'site1_ul', 'site2_ul', 'site3_ul']]
with open("output_svod.csv", encoding='cp1251') as f_obj:
    csv_dict_reader(f_obj)
       

In [0]:
#import ssl

#try:
#    _create_unverified_https_context = ssl._create_unverified_context
#except AttributeError:
    # Legacy Python that doesn't verify HTTPS certificates by default
#    pass
#else:
    # Handle target environment that doesn't support HTTPS verification
#   ssl._create_default_https_context = _create_unverified_https_context

In [0]:
def site_score(d):
    global data
    
    res = []
    for k in range(2,5):
        url = d[k]
        
        if k > 2 and (d[k].strip() == '' or d[k].strip() == d[k-1].strip()):
            res.append([])
            continue
            
        if k == 3 and (not '.' in url or 'html' in url or '?' in url or '.php' in url):
            url = d[2] + '/' + url

        print(url)

        if not url.startswith('http'):
            url = f'http://{url}'

        resp = None
        try:
            resp = requests.get(url, timeout = 10)
        except:
            res.append([])
            continue

        # здесь должен быть break
        # resp.status_code


        orgname_rate = 0
        orgname_rate2 = 0
        addr_rate = 0
        last_adr_word = 0

        for w in d[5]: #  сокращенное наименование организации
          # print(w)
          x = re.findall(w, resp.text.upper())
          # print(x)
          orgname_rate += len(x)
        # print(orgname_rate)

        for w in d[6]: #  описание организации
          x = re.findall(w, resp.text.upper())
          orgname_rate2 += len(x)

        for w in d[7]: #  адрес организации
          x = re.findall(w, resp.text.upper())
          last_adr_word = len(x)
          addr_rate += len(x)
          
        res.append([orgname_rate, orgname_rate2, addr_rate, last_adr_word])
          
    return res
 

In [30]:
import time
print(time.ctime())

for N in range(1, len(data)):
    print(data[N])
    x = site_score(data[N])

    data[N] = data[N] + x + [sum(x[0])]  + [sum(x[1])]  + [sum(x[2])]
    data[N] += [x[0][2] if x[0] else 0] + [x[1][2] if x[1] else 0] + [x[2][2] if x[2] else 0]
    data[N] += [x[0][3] if x[0] else 0] + [x[1][3] if x[1] else 0] + [x[2][3] if x[2] else 0]

    csv_writer(data, "output_site.csv")
    
print(time.ctime())

Sat Sep 21 13:48:29 2019
['7447010227', 'АО "ЧЭМК"', 'chemk.ru', 'chemk.ru', 'http://kilmz.ru', ['ЧЭМК'], ['ЧЕЛЯБИНСКИЙ', 'ЭЛЕКТРОМЕТАЛЛУРГИЧЕСКИЙ'], ['ЧЕЛЯБИНСК', 'ТАНКОГРАДА']]
chemk.ru
http://kilmz.ru
['7327012462', 'ПАО "Ульяновскэнерго"', 'ulenergo.ru', 'ulenergo.ru', 'ulenergo.ru', ['УЛЬЯНОВСКЭНЕРГО'], [], ['УЛЬЯНОВСК', 'СТАНКОСТРОИТЕЛЕЙ']]
ulenergo.ru
['711008455', 'АО "Каббалкэнерго"', 'kabbalkenergo.ru', 'kabbalkenergo.ru', 'www.kabene.elektra.ru/', ['КАББАЛКЭНЕРГО'], ['МЕЖРЕГИОНАЛЬНАЯ'], ['НАЛЬЧИК', 'ЩОРСА']]
kabbalkenergo.ru
www.kabene.elektra.ru/
['901000327', 'АО "Карачаево-Черкесскэнерго"', 'kchenergo.ru', '', '#Н/Д', ['КАРАЧАЕВО-ЧЕРКЕССКЭНЕРГО'], [], []]
kchenergo.ru
#Н/Д
['1502002701', 'АО "Севкавказэнерго"', 'sevkavenergo.ru', 'sevkavenergo.ru', 'www.vladi.elektra.ru/', ['СЕВКАВКАЗЭНЕРГО'], [], ['ВЛАДИКАВКАЗ', 'ТАМАЕВА']]
sevkavenergo.ru
www.vladi.elektra.ru/
['6315222985', 'ПАО "Самараэнерго"', 'samaraenergo.ru', 'samaraenergo.ru', 'www.aosamen.elektra.ru/', ['САМАРАЭ

In [0]:
N

In [0]:
a = 'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "РЕГИОНЫ - ЭНЕРГО"\
РУКОВОДИТЕЛЬ: КОЛЕСНИКОВА ЕКАТЕРИНА НИКОЛАЕВНА\
ИНН: 5024170498\
АДРЕС: 143402, МОСКОВСКАЯ ОБЛ, КРАСНОГОРСК Г, ЗНАМЕНСКАЯ УЛ, ДОМ 5'

parse_orgname(a)

In [0]:
q =' 394029, ВОРОНЕЖСКАЯ ОБЛ, ВОРОНЕЖ Г, МЕРКУЛОВА УЛ, 7, А'

parse_orgname([q])

NameError: ignored

In [0]:
xx = []
for d in data:
  x = re.findall(r'(\.\w*)$', d[4])
  xx.append(x[0] if x else '')
  
set(xx)

{'', '.com', '.html', '.net', '.org', '.ru', '.su', '.рф'}

In [0]:
d5=[]
d6=[]
for d in data[1:]:
  for dd in d[5]:
    if len(dd) < 3:
      d5.append(dd)
      print(dd, d[5])
  for dd in d[6]:
    if len(dd) < 3:
      d6.append(dd)
      print(dd, d[6])
      
d5 = set(d5)
d6 = set(d6)
print(d5)
print(d6)

set()
set()


In [0]:
for d in data:
  print(d)

['INN', 'Name', 'site1', 'site2', 'site_list_org', 'orgname_keywords', 'orgname_keywords2', 'addr_keywords']
['7447010227', 'АО "ЧЭМК"', 'chemk.ru', 'chemk.ru', 'http://kilmz.ru', ['ЧЭМК'], ['ЧЕЛЯБИНСКИЙ', 'ЭЛЕКТРОМЕТАЛЛУРГИЧЕСКИЙ'], ['ЧЕЛЯБИНСК', 'ТАНКОГРАДА'], [4, 4, 5], [4, 4, 5], [1, 0, 2]]
['7327012462', 'ПАО "Ульяновскэнерго"', 'ulenergo.ru', 'ulenergo.ru', 'ulenergo.ru', ['УЛЬЯНОВСКЭНЕРГО'], ['УЛЬЯНОВСКЭНЕРГО'], ['УЛЬЯНОВСК', 'СТАНКОСТРОИТЕЛЕЙ'], [3, 3, 8], [3, 3, 8], [3, 3, 8]]
['711008455', 'АО "Каббалкэнерго"', 'kabbalkenergo.ru', 'kabbalkenergo.ru', 'www.kabene.elektra.ru/', ['КАББАЛКЭНЕРГО'], ['МЕЖРЕГИОНАЛЬНАЯ'], ['НАЛЬЧИК', 'ЩОРСА'], [18, 0, 4], [18, 0, 4], 'bad url']
['901000327', 'АО "Карачаево-Черкесскэнерго"', 'kchenergo.ru', '', '#Н/Д', ['КАРАЧАЕВО-ЧЕРКЕССКЭНЕРГО'], [], [], [19, 0, 0], [19, 0, 0], 'bad url']
['1502002701', 'АО "Севкавказэнерго"', 'sevkavenergo.ru', 'sevkavenergo.ru', 'www.vladi.elektra.ru/', ['СЕВКАВКАЗЭНЕРГО'], ['СЕВКАВКАЗЭНЕРГО'], ['ВЛАДИКАВКАЗ', 'Т